In [7]:
import flask
import googleapiclient.discovery
import numpy as np
import pandas as pd
import os, json
from numpy import array
from google.api_core.client_options import ClientOptions
from flask import Flask, request, jsonify

In [8]:
df=pd.read_csv('/home/nusa/Documents/ProjectBangkit2021/data/Merged3.1.csv')

In [9]:
df['tanggal'] = pd.to_datetime(df['tanggal'])
df.drop('critical', axis=1, inplace=True)
df.drop('categori', axis=1, inplace=True)
df.drop('lokasi_spku', axis=1, inplace=True)
df.set_index('tanggal', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4088 entries, 2010-01-01 to 2021-03-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pm10    4088 non-null   int64  
 1   so2     4088 non-null   int64  
 2   co      4088 non-null   float64
 3   o3      4088 non-null   int64  
 4   no2     4088 non-null   int64  
 5   max     4088 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 223.6 KB


In [20]:
data = df.values[-7:]
data = np.expand_dims(data, axis=0)
data

array([[[38., 41.,  9., 41., 17., 68.],
        [57., 42., 19., 53., 28., 83.],
        [62., 49., 13., 63., 33., 91.],
        [57., 43., 15., 71., 31., 78.],
        [65., 48., 19., 67., 37., 91.],
        [59., 42., 15., 55., 30., 76.],
        [56., 43., 18., 48., 25., 69.]]])

In [3]:
def predict_json(project, region, model, instances, version=None):
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/nusa/Documents/euphoric-fusion-312609-e20f53f87718.json"

    prefix = "{}-ml".format(region) if region else "ml"
    api_endpoint = "https://{}.googleapis.com".format(prefix)
    client_options = ClientOptions(api_endpoint=api_endpoint)
    service = googleapiclient.discovery.build(
        'ml', 'v1', client_options=client_options)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [4]:
app = flask.Flask(__name__)
app.config["DEBUG"] = True

In [5]:
@app.route('/', methods=["GET"])
def index():
    project = request.args.get('project')
    region = request.args.get('region')
    model = request.args.get('model')
    instances = data
    version = request.args.get('version')
    inverse=np.array([178.,  72., 134., 314.,  63.])
    x = predict_json(project, region, model, instances, version)
    x = x*inverse
    x = x.round(2)
    pm10, so2, co, o3, no2 = np.dsplit(x, 5)
    pollutant = {}
    pollutant['pm10'] = pm10[0][1].tolist()
    pollutant['so2'] = so2[0][1].tolist()
    pollutant['co'] = co[0][1].tolist()
    pollutant['o3'] = o3[0][1].tolist()
    pollutant['no2'] = no2[0][1].tolist()
    pollutant = jsonify(pollutant)
    return pollutant

In [6]:
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/home/nusa/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
